I want to output words frequency.I already pro-processed texts and use directly the lemmad text

In [2]:
import re
import pandas as pd
from collections import Counter

In [3]:
from pathlib import Path

# Define the relative file path
file_path = Path("./deepseek_output_lemmas.txt")

# Open the file using Path
with file_path.open("r", encoding="utf-8") as f:
    text = f.read()

# Print data type and first 20 characters
print(type(text))  # Check data type
print(text[:20])  # Print first 20 characters


<class 'str'>
uk
able
resist
china


In [4]:
text = text.split() #convert string to list,It splits the string into a list of words, using whitespace
print(type(text))  # Check data type
print(text[:20])

<class 'list'>
['uk', 'able', 'resist', 'china', 'tech', 'dominance', 'china', 'success', 'technology', 'come', 'thin', 'air', 'give', 'unlikely', 'origin', 'deepseek', 'deep', 'shock', 'obscure', 'hangzhou']


In [5]:
# Count word frequencies
word_freq = Counter(text)

# Convert to DataFrame for easy viewing
df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
df = df.sort_values(by='Frequency', ascending=False)

# Display top 20 words
df.head(20)


,Word,Frequency
14,deepseek,606
41,ai,405
114,company,305
243,model,305
213,say,301
3,china,287
52,chinese,262
4,tech,219
288,chip,150
55,like,146


What can we do with the words frequency?In my practical process, I check words frequecy on voyant tool and antconc, and when I check it in voyant tool, I see words with green and red color, later I figruer out the color means sentiment,green means positive and red means negative,This inspired me to further research: what positive words and negative words the media use for discribing "deepseek"? and the paper"history text"also inspired me.

In [9]:
pip install openpyxl



[notice] A new release of pip is available: 25.0 -> 25.1.1
[notice] To update, run: /usr/local/anaconda3/envs/numpy_env/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


I want to export the word frequency table to an Excel file using Pandas so I can manually edit it later.

In [11]:
# Convert to DataFrame
df = pd.DataFrame(word_freq.items(), columns=['Word', 'Frequency'])
df = df.sort_values(by='Frequency', ascending=False)

# Export to Excel
df.to_excel("word_frequencies.xlsx", index=False, engine='openpyxl')

print("Excel file 'word_frequencies.xlsx' has been saved!")


Excel file 'word_frequencies.xlsx' has been saved!


get sentiment score for each word, so we can view the words with sentiment and frequency the same time.we apply sentiment analysis to each word,polarity score ranges from -1 to 1, and remove newtral words(score=0).

In [14]:
import pandas as pd
from textblob import TextBlob

# Load word frequency table from Excel
df = pd.read_excel("word_frequencies.xlsx")

# Define a function to get sentiment scores using TextBlob
def get_sentiment(word):
    score = TextBlob(word).sentiment.polarity  # Polarity score ranges from -1 to 1
    return score

# Apply sentiment analysis to each word
df["Sentiment Score"] = df["Word"].apply(get_sentiment)

# Remove neutral words (score = 0)
df = df[df["Sentiment Score"] != 0]

# Sort by sentiment score
df = df.sort_values(by="Sentiment Score", ascending=False)

# Export the filtered table to a new Excel file
df.to_excel("sentiment_word_frequencies_textblob.xlsx", index=False, engine="openpyxl")

print("Filtered sentiment table saved as 'sentiment_word_frequencies_textblob.xlsx'!")


TypeError: The `text` argument passed to `__init__(text)` must be a string, not <class 'float'>

In [ ]:
#let's visualize the result
import matplotlib.pyplot as plt

# Select top 10 positive and negative words
top_pos = df.nlargest(10, "Sentiment Score")
top_neg = df.nsmallest(10, "Sentiment Score")

# Plot
plt.figure(figsize=(10, 3))
plt.barh(top_pos["Word"], top_pos["Sentiment Score"], color="green", label="Positive")
plt.barh(top_neg["Word"], top_neg["Sentiment Score"], color="red", label="Negative")
plt.xlabel("Sentiment Score")
plt.ylabel("Word")
plt.title("Top Positive and Negative Words")
plt.legend()
plt.show()


Here, we have to evaluate the result. Since machine aotomatly filtered a set of words, so when we look back to the words frequency table, some wimportant words are missing.like "innovation""create""ban".
as we can see clearly,"shocking" and "shockingly", the machine give them -1.0, means extremely negative, but in the context, we are not sure it's a negative word, for deepseek is defnertly a extremely possitive word with score +1.0, so this demonstrante that we can not rely on the tool analysis completely.And as I interprete, I consider the phnomonon is a breakthrough so I assign manually the score +1.0 to "shocking", and I combine the frequency of two words "shockingly""shocking"'. and words frequency also matters, so innovation(freqency47)is also added  to the sentiment analysis table,and I assign the score 1.0 to the word "innovation".added "ban" (frequency 40) with score -0.8.starup(47)with score0.3.lead(77) with score 0.2,create(42)score0.7, there are conducted manually.the modified table was restored with the name"sentiment_word_frequencies_textblob_man.xlsx".

I am thinking about delete some words which with low frequency and low scores, since too tired doing it manually. why frequecy below 20?I visualize the whole data first, and notice that the scale of words with frequecy above 20 is large enough.so I set 20.

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("./sentiment_word_frequencies_textblob_man.xlsx", engine="openpyxl")

# Ensure numeric values for filtering
df["Frequency"] = pd.to_numeric(df["Frequency"], errors="coerce")
df["Sentiment Score"] = pd.to_numeric(df["Sentiment Score"], errors="coerce")

# Remove words with frequency < 20 and sentiment score < 0.5
df_filtered = df[(df["Frequency"] >20) | (df["Sentiment Score"] >= 0.5)]

# Save the cleaned data to a new file
df_filtered.to_excel("./positive_sentiment_words.xlsx", index=False)

print("✅ Done! The filtered words are saved in 'positive_sentiment_words.xlsx'.")


after filtering It's still has 81 words left,it's still too much for viralization.I change some score manually, the aim is to show them , I think these adjectiv words is valuable, so I change the score from0.5 to 0.6, so that I can adjust parament to 0.6(higher). I also combine some words which have the same root:"innovative""innovation","develop"and"development","success""successful". Adjust parament is a hard work, and I spen a lot of time on this step, the aim is to filter the most valuble data, but what is valuable, apparently the precess mixed with my subjectivty.I also add words like"breakthrough"(frequency 30),power(79),raise(27),cheap(28)
simutaniously, I delete some words which has high score but meaningless in this contex, like"honest",frequency just once ,and score 0.6, I don't want show it so delete it.and score of"welcome" is 0.8, when it's done, we have to restore the result as"positive_sentiment_words_man", otherwise when we run the code again, the excle file will rewrite.
Firstly, we consider frequency and sentiment score these two weight, so we want to show the words in two dimentional space. And secondly we will show the words with high sentiment score.So in this first table, we just consider words with frequency above 20 and sentiment score above0.6.

We need check the result and delete some meaningless words manally,now the results reduced to 28 words, and we can visualize the result.when we check the table, there are many words have the same score, it leads to the problem that these points will be concentrated together and difficult to distinguish.so we manually chage the score slittly

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read data from Excel file
file_path = './positive_sentiment_words_man.xlsx'  
df = pd.read_excel(file_path)

# Create a scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(df['Frequency'], df['Sentiment Score'])

# Add labels to each point with offset
for i, row in df.iterrows():
    offset_x = np.random.uniform(-1, 1)
    offset_y = np.random.uniform(-0.02, 0.02)
    plt.text(row['Frequency'] + offset_x, row['Sentiment Score'] + offset_y, row['Word'], fontsize=6, ha='center', va='bottom')

# Set plot title and axis labels
plt.title('Word Frequency vs Sentiment Score')
plt.xlabel('Frequency')
plt.ylabel('Sentiment Score')

# Display the plot
plt.show()

We try different visualization,

In [ ]:
import matplotlib.pyplot as plt

# Read data from Excel file
file_path = './positive_sentiment_words_man.xlsx' 
df = pd.read_excel(file_path)

# Sort by Sentiment Score (descending) and take top 20 words
df_top = df.sort_values(by="Sentiment Score", ascending=False).head(20)

# Plot bar chart
plt.figure(figsize=(12, 6))
plt.barh(df_top["Word"], df_top["Sentiment Score"], color="skyblue")
plt.xlabel("Sentiment Score")
plt.ylabel("Word")
plt.title("Top 20 Positive Words with Highest Sentiment Scores")
plt.gca().invert_yaxis()  # Invert Y-axis for better readability
plt.grid(axis="x", linestyle="--", alpha=0.7)

# Show the plot
plt.show()

Now let's do the negative words part.

In [ ]:
import pandas as pd

# Load the Excel file
df = pd.read_excel("./sentiment_word_frequencies_textblob_man.xlsx", engine="openpyxl")

# Ensure numeric values for filtering
df["Frequency"] = pd.to_numeric(df["Frequency"], errors="coerce")
df["Sentiment Score"] = pd.to_numeric(df["Sentiment Score"], errors="coerce")

# Filter the data
df_filtered = df[((df["Frequency"] > 20) | (df["Sentiment Score"] < - 0.5)) & (df["Sentiment Score"] <= 0)]

# Save the cleaned data to a new file
df_filtered.to_excel("./negative_sentiment_words.xlsx", index=False)

print("✅ Done! The filtered words are saved in 'negative_sentiment_words.xlsx'.")


the result has 20 words, so it's possiable for virualization.I also add some words which I considered are meaningful.like"risk"(frequency22),"security"(frequency31),rival(34),and we saved it as "2negative_sentiment_words.xlsx"

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Read data from Excel file
file_path = './2negative_sentiment_words.xlsx'  
df = pd.read_excel(file_path)

# Create a scatter plot
plt.figure(figsize=(12, 8))
plt.scatter(df['Frequency'], df['Sentiment Score'])

# Add labels to each point with offset
for i, row in df.iterrows():
    offset_x = np.random.uniform(-1, 1)
    offset_y = np.random.uniform(-0.02, 0.02)
    plt.text(row['Frequency'] + offset_x, row['Sentiment Score'] + offset_y, row['Word'], fontsize=6, ha='center', va='bottom')

# Set plot title and axis labels
plt.title('Word Frequency vs Sentiment Score')
plt.xlabel('Frequency')
plt.ylabel('Sentiment Score')

# Display the plot
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Read data from Excel file
file_path = './2negative_sentiment_words.xlsx' 
df = pd.read_excel(file_path)

# Sort by Sentiment Score (ascending) and take top 20 words
df_top = df.sort_values(by="Sentiment Score", ascending=True).head(20)

# Plot bar chart
plt.figure(figsize=(12, 6))
plt.barh(df_top["Word"], df_top["Sentiment Score"], color="red")
plt.xlabel("Sentiment Score")
plt.ylabel("Word")
plt.title("Top 20 negative Words with lowest Sentiment Scores")
plt.gca().invert_yaxis()  # Invert Y-axis for better readability
plt.grid(axis="x", linestyle="--", alpha=0.7)

# Show the plot
plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the positive words Excel file
positive_file_path = './2positive_sentiment_words.xlsx'
positive_df = pd.read_excel(positive_file_path)

# Read the negative words Excel file
negative_file_path = './2negative_sentiment_words.xlsx'  
negative_df = pd.read_excel(negative_file_path)

# Convert 'Sentiment Score' to numeric (force errors to NaN) and drop NaN values for positive words
positive_df["Sentiment Score"] = pd.to_numeric(positive_df["Sentiment Score"], errors='coerce')
positive_df = positive_df.dropna(subset=["Sentiment Score"])

# Convert 'Sentiment Score' to numeric (force errors to NaN) and drop NaN values for negative words
negative_df["Sentiment Score"] = pd.to_numeric(negative_df["Sentiment Score"], errors='coerce')
negative_df = negative_df.dropna(subset=["Sentiment Score"])

# Plot the combined chart
plt.figure(figsize=(10, 6))

# Plot positive sentiment words
plt.barh(positive_df["Word"][-10:], positive_df["Sentiment Score"][-10:], color="green", label="Positive")

# Plot negative sentiment words
plt.barh(negative_df["Word"][:10], negative_df["Sentiment Score"][:10], color="red", label="Negative")

plt.xlabel("Sentiment Score")
plt.ylabel("Word")
plt.title("Top Positive and Negative Words with Sentiment Scores")
plt.legend()
plt.grid(True)
plt.show()